In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
import sklearn
import random
import torchvision
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# setting seed
torch.manual_seed(1)
if device == "cuda":
    torch.cuda.manual_seed_all(1)
random.seed(1)

In [4]:
# set data path
train_dir_path = "./2021-ai-w10-p2/images/images"
test_dir_path = "./2021-ai-w10-p2/test_data/test_data"
submission_csv_path = "./2021-ai-w10-p2/format.csv"

In [5]:
# load train data 
from tqdm import tqdm
import os
from PIL import Image
def load_train_data(dir_path):
    train_y = []
    train_x = []
    for dir_name in tqdm(os.listdir(dir_path)):
        for img_name in os.listdir(os.path.join(dir_path,dir_name)):
            img_path = os.path.join(dir_path,dir_name,img_name)
            img = np.array(Image.open(img_path)).transpose((2,0,1))
            # Scaling
            img = img/np.max(img)
            
            train_x.append(img)
            train_y.append(int (dir_name))
    return np.array(train_x),np.array(train_y)

train_x, train_y  = load_train_data(train_dir_path)
print(train_x.shape, train_y.shape)

100%|██████████| 10/10 [00:11<00:00,  1.17s/it]


(50000, 3, 32, 32) (50000,)


In [6]:
# set data on Tensor and Dataset
train_x = torch.Tensor(train_x).to(device)
train_y = torch.LongTensor(train_y).to(device)
train_dataset = torch.utils.data.TensorDataset(train_x,train_y)

In [7]:
model = torchvision.models.googlenet(pretrained=True)

# freezing
for param in model.parameters():
    param.requires_grad = False

model.fc = torch.nn.Linear(in_features=1024, out_features=10, bias=True).to(device)

torch.nn.init.xavier_normal_(model.fc.weight)

model = model.to(device)
print(model)

c:\python39\lib\site-packages\torchvision\models\googlenet.py:77: FutureWarning: The default weight initialization of GoogleNet will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn('The default weight initialization of GoogleNet will be changed in future releases of '


GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [8]:
# setting param, optim, cost function,dataloader
from torch.utils.data import DataLoader

lr = 1e-3
batch_size = 100
optim = torch.optim.Adam(model.parameters(), lr=lr)
train_data_loader = DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle=True)
epochs = 40
loss = torch.nn.CrossEntropyLoss()

In [9]:
def train_model(model,data_loader,optim):
    sum_cost = 0
    for x,y in data_loader:
        output = model(x)
        cost = loss(output,y)
        
        optim.zero_grad()
        cost.backward()
        optim.step()
        sum_cost += cost.item()
    return sum_cost

In [10]:
# learning
model.train()
for epoch in tqdm(range(epochs+1)):
    cost = train_model(model,train_data_loader,optim)
    cost = cost/len(train_data_loader.dataset)
    if epoch % (epochs/10) == 0 :
#         model.eval()
#         predict = model(train_x)
#         predict = predict
#         predict = torch.argmax(predict,dim=1)
#         acc = predict == train_y
#         acc = acc.float().mean().cpu().detach().item()
        print(epoch,cost)

  0%|          | 0/101 [00:00<?, ?it/s]c:\python39\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
  1%|          | 1/101 [00:06<10:33,  6.34s/it]

0 0.0034293420362472536


 11%|█         | 11/101 [00:50<06:41,  4.47s/it]

10 0.0007323569065332412


 21%|██        | 21/101 [01:35<06:01,  4.51s/it]

...

40 9.503438141196966e-05


 50%|█████     | 51/101 [03:50<03:42,  4.46s/it]

50 7.16200646571815e-05


 52%|█████▏    | 53/101 [04:01<03:38,  4.56s/it]


KeyboardInterrupt: 

In [11]:
with torch.no_grad():
    model.eval()
    predict = model(train_x)
    predict = torch.argmax(predict,dim=1)
    acc = train_y == predict
    acc = acc.float().mean().cpu().detach().item()
print(acc)

0.9742199778556824


In [12]:
# load test data 
def load_test_data(dir_path):
    test_x = []
    img_names = os.listdir(dir_path)
    img_names = [int(img_name.split(".")[0]) for img_name in img_names]
    sorted_img_names = sorted(img_names)
    for img_name in tqdm(sorted_img_names):
        img_path = os.path.join(dir_path,str(img_name)) + ".png"
        img = np.array(Image.open(img_path)).transpose((2,0,1))
        # Scaling
        img = img/np.max(img)
        test_x.append(img)
    return np.array(test_x)

test_x = load_test_data(test_dir_path)
print(test_x.shape)

100%|██████████| 10000/10000 [00:02<00:00, 4086.64it/s]


(10000, 3, 32, 32)


In [13]:
# setd test data on Tensor and Dataset
test_x = torch.Tensor(test_x).to(device)
test_dataset = torch.utils.data.TensorDataset(test_x)
test_data_loader = torch.utils.data.DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle=False)

In [18]:
def test_model(model,data_loader):
    predicts = []
    for x in tqdm(data_loader):
        predict = model(x[0])
        predict = torch.argmax(predict,dim=1)
        predicts += np.array(predict.cpu().detach()).tolist()
    return np.array(predicts)

In [20]:
# submit
submission = pd.read_csv(submission_csv_path)
with torch.no_grad():
    model.eval()
    predict = test_model(model,test_data_loader)
    submission['label'] = predict
submission.to_csv("submission.csv",index=False)

100%|██████████| 20/20 [00:00<00:00, 37.69it/s]
